# Team EGE - BLG453E Term Project
Egehan Orta - 150160124

Yavuz Ege Okumuş - 150160118
## Mine Game

In [ ]:
import pyautogui as pg
import cv2 as cv
import dlib
import numpy as np
import os
import matplotlib.pyplot as plt
import time

In [ ]:
my, mx = 270,960
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('./shape_predictor_68_face_landmarks.dat')

In [ ]:
gaussian_blur = np.array([[1,2,1],
                         [2,4,2],
                         [1,2,1]])/64
sharpen = np.array([[0,-1,0],
                   [-1,5,-1],
                   [0,-1,0]])

In [ ]:
def calc_area(m):
    m = m.reshape((-1,2))
    ma = np.max(m,axis=0)
    mi = np.min(m,axis=0)
    return np.product(ma-mi)

In [ ]:
def img_process(src):
    image = src.copy()
    image = image[0:540, mx-270:mx+270]
    
    gray = cv.cvtColor(image, cv.COLOR_RGB2GRAY)
    outside = np.where(gray==154)
    blur = cv.filter2D(gray, -1, gaussian_blur)
    sharp = cv.filter2D(blur, -1, sharpen)
    high_thresh, thresh = cv.threshold(sharp,220,255,cv.THRESH_BINARY + cv.THRESH_OTSU)
    
    thresh[outside[0], outside[1]] = 154
    temp = thresh[200:340, 220:320].copy()
    temp = cv.blur(temp, (3,3))
    temp = cv.blur(temp, (3,3))
    temp = cv.blur(temp, (3,3))
    
    opening = cv.morphologyEx(temp, cv.MORPH_OPEN, np.ones((3,3)), iterations=1)
    close = cv.morphologyEx(opening, cv.MORPH_CLOSE, np.ones((3,3)), iterations=15)

    cl = thresh.copy()
    cl[200:340, 220:320] = close
    close = cl.copy()
    
    canny = cv.Canny(close, 0.5*high_thresh, high_thresh)
    canny = cv.dilate(canny, np.ones((3,3)), iterations = 3)
    _, __ = canny.shape
    canny = cv.rectangle(canny,(0,0),(_-1,__-1),255,15)
    
    cnts = cv.findContours(canny, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    min_area = (270**2)*0.4
    max_area = (270**2)*1.2
    colors = [(255,0,0), (0,255,0), (0,0,255), (255,0,255), (0,255,255)]
    image_number = 0
    for c in cnts[0]:
        area = calc_area(c)
        if (area > min_area and area < max_area) or False:
            x,y,w,h = cv.boundingRect(c)
            ROI = image[y:y+h, x:x+h]
            img2 = cv.rectangle(image, (x, y), (x + w, y + h), colors[image_number%5], -1)
            image_number += 1
    
    return img2

In [ ]:
def recolor(image):
    img = image.copy()
    color = img[270,270]
    possible = None
    if tuple(color) not in [(255,0,0), (0,255,0), (0,0,255), (255,0,255), (0,255,255)]:
        for px in range(10):
            cl = tuple(img[270,270-px])
            if cl in [(255,0,0), (0,255,0), (0,0,255), (255,0,255), (0,255,255)]:
                possible=(cl,px)
                break
        for px in range(10):
            cl = tuple(img[270,270+px])
            if cl in [(255,0,0), (0,255,0), (0,0,255), (255,0,255), (0,255,255)]:
                if possible is not None:
                    if possible[1] > px:
                        possible=(cl,px)
                        break
                    else:
                        break
                else:
                    possible=(cl,px)
                    break
        for px in range(10):
            cl = tuple(img[270-px,270])
            if cl in [(255,0,0), (0,255,0), (0,0,255), (255,0,255), (0,255,255)]:
                if possible is not None:
                    if possible[1] > px:
                        possible=(cl,px)
                        break
                    else:
                        break
                else:
                    possible=(cl,px)
                    break
        for px in range(10):
            cl = tuple(img[270+px,270])
            if cl in [(255,0,0), (0,255,0), (0,0,255), (255,0,255), (0,255,255)]:
                if possible is not None:
                    if possible[1] > px:
                        possible=(cl,px)
                        break
                    else:
                        break
                else:
                    possible=(cl,px)
                    break
        color = np.array(possible[0])      
        
    current = np.where((img[:,:] == color).all(axis=2))
    img[current[0], current[1]] = [255,255,0] #Colorize current box yellow
    
    clean = np.where((img[:,:] != [255,255,0]).any(axis=2))
    if clean:
            img[clean[0], clean[1], :] = [0,0,0]
            
    current_mid = int(np.mean(current[0])) , int(np.mean(current[1]))
    
    t_border = np.min(current[0])+54
    b_border = np.max(current[0])-54
    l_border = np.min(current[1])+54
    r_border = np.max(current[1])-54
    
    mid_y_t = max(current_mid[0]-20, 0)
    mid_y_b = min(current_mid[0]+20, 540)
    mid_x_l = max(current_mid[1]-20, 0)
    mid_x_r = min(current_mid[1]+20, 540)
    
    img[mid_y_t:mid_y_b, mid_x_l:mid_x_r] = [255,0,0]
    
    img[t_border-45:t_border, l_border-45:r_border+45] = [0,255,0]
    img[b_border:b_border+45, l_border-45:r_border+45] = [0,255,0]
    img[t_border-45:b_border+45, l_border-45:l_border] = [0,255,0]
    img[t_border-45:b_border+45, r_border:r_border+45] = [0,255,0]
    
    return img

In [ ]:
class grid():
    def __init__(self):
        self.info = dict(u = True, r = True ,d = True, l = True)

In [ ]:
def state():
    img = np.array(pg.screenshot())
    gray = cv.cvtColor(img, cv.COLOR_RGB2GRAY)
    outside = np.where(gray==154)
    thresh = cv.threshold(gray,32,255,cv.THRESH_BINARY)[1]
    thresh = cv.dilate(thresh, np.ones((5,5)))
    thresh[outside[0], outside[1]] = 154
    
    uniq = np.unique(thresh[my-55:my+55, mx-32:mx+28], return_counts=True)
    if uniq[0][0] == 0 and uniq[1][0] > 2000:
        thresh[my-55:my+55, mx-32:mx+28] = 0
    return img, thresh

In [ ]:
def isgood(img):
    gray = cv.cvtColor(img, cv.COLOR_RGB2GRAY)
    p = predictor(gray, detector(gray)[0])
    if p.parts()[9].y < 1060:
        return False
    return True

In [ ]:
def move(m, s):
    if m == 'u':
        pg.keyDown('w')
        time.sleep(s)
        pg.keyUp('w')
    elif m == 'r':
        pg.keyDown('d')
        time.sleep(s)
        pg.keyUp('d')
    elif m == 'd':
        pg.keyDown('s')
        time.sleep(s)
        pg.keyUp('s')
    elif m == 'l':
        pg.keyDown('a')
        time.sleep(s)
        pg.keyUp('a')
    time.sleep(1)

In [ ]:
def invmove(m):
    i = 'uldr'
    j = 'drul'
    
    return j[i.index(m)]

In [ ]:
def isvalid(m, thresh):
    if m == 'u':
        return thresh[my-270, mx] != 154
    elif m == 'r':
        return thresh[my, mx+270] != 154
    elif m == 'd':
        return thresh[my+270, mx] != 154
    elif m == 'l':
        return thresh[my, mx-270] != 154

In [ ]:
def snap():
    return np.array(pg.screenshot())

In [ ]:
time.sleep(5)
cur = grid()
force = False
notcheck = ''
while True:
    img, thresh = state()
    for i in 'urdl':
        if i == notcheck:
            continue
        cur.info[i] = isvalid(i,thresh)
    notcheck = ''
    if not force:
        text = 'urdl'
    else:
        force = False
    for idx,i in enumerate(text):
        if cur.info[i]:
            reimg = recolor(img_process(snap()))

            while (reimg[270,270] != [0,255,0]).any():
                move(i,0.15)
                reimg = recolor(img_process(snap()))
            if not isgood(snap()):
                
                reimg = recolor(img_process(snap()))
                while (reimg[270,270] != [255,0,0]).any():
                    move(invmove(i), 0.15)
                    reimg = recolor(img_process(snap()))
                
                if not isgood(snap()):
                    while (reimg[270,270] != [0,255,0]).any():
                        move(i,0.2)
                        reimg = recolor(img_process(snap()))
                    while (reimg[270,270] != [255,0,0]).any():
                        move(invmove(i), 0.15)
                        reimg = recolor(img_process(snap()))
                    force = True
                    text = 'urdl'[idx+1:]
                    break
                else:
                    cur.info[i] = False
                
            else:
                reimg = recolor(img_process(snap()))
                while (reimg[270,270] != [255,0,0]).any():
                    move(i, 0.15)
                    reimg = recolor(img_process(snap()))
                cur = grid()
                cur.info[invmove(i)] = False
                notcheck = invmove(i)
                break

    
    if not any(cur.info.values()):
        break